In [43]:
import numpy as np
import pandas as pd
from pythainlp.tokenize import word_tokenize
from pythainlp.spell import correct
import csv

In [44]:
word_file = pd.read_csv("text_test.csv")

#Negative keyword
neg_pd = pd.read_csv("neg.txt", sep = "/n", header = None)

neg = np.array(neg_pd)

#Neutral keyword
neutral_pd = pd.read_csv("neutral.txt", sep = "/n", header = None)

neutral = np.array(neutral_pd)

#Positive keyword
pos_pd = pd.read_csv("pos.txt", sep = "/n", header = None)

pos = np.array(pos_pd)

In [45]:
#Create array and dictionary

word_file_2 = np.array(word_file)

word_file_3 = dict(word_file_2)

print(word_file_3)

{'Worawut Wutpradis\xa0': 'เฟซปลอม โกงมือถือ แจ้งระงับไป มันบอกไม่ผิดกฏ คนด่าหยาบคายไม่ผิด พอเราจะประจานมัน เฟซมันบอกเราผิดกฏจะโดนระงับบัญชี\xa0', 'Hathai Chanyanupap\xa0': 'Chuda Lee อ่านได้2แผ่น แล้วเหนื่อย แต่คิดว่าดี55555\xa0', 'Rinma Linma Malin\xa0': 'Cho S น้องสาวอ่านอันนี้คับบบ\xa0', 'Porntep Seedang': 'สวัสดีครับ ขอสอบถามหน่อยครับ อยากให้ ในเว็บ โชว ของต่ละวันทำได้ไหมครับ', 'Bumbim Poonphisa': 'เกี่ยวกับการตลาด การยิงโฆษณา ใน facebook ค่ะ', 'Donuttw Pinsuwan': 'สวัสดีครับ ชื่อ นัท ณัฐวุฒิ ปิ่นสุวรรณเบอร์ติดต่อ 082-883-8249ขอสอบถามเรื่องสมัครงานครับ ตอนนี้มีตำแหน่งงาน AE , Digital Maketing ว่างไหมครับ สนใจสมัครตำแหน่งนี้ครับ ขอบคุณครับ', 'ฅนหัวโบราณ ฅนหัวโบราณ': 'ขอทราบรายละเอียดหน่อยครับ', 'Nisara Mini\xa0': 'Nitjawan Wianglo แน่นหน่อยแต่มีประโยชน์ เหมือนเบียร์อ่ะแหละ\xa0', 'Vajirawich Wongpuvarak\xa0': 'Ae Viriyasiri ยาวไปอะ เก็วโควต้าอ่านหนังสือสวดมนต์\xa0', 'Pairpear PairPear\xa0': 'Nat Nathaniche นัดได้วันนี้ว่าง lol\xa0', 'Chalatip Sakonchaijaroen': 'สร้อย0817919165', 'La

In [40]:
#Tokenizer and add to list of keyword

list_of_list_word = []

for file_sentence in word_file_2:
    token_word = word_tokenize(file_sentence[1])
    list_word = []

    for word in token_word:
        if word.isspace():
            continue
        else:
            new_word = correct(word)
            list_word.append(new_word)

    list_of_list_word.append(list_word)

In [59]:
#Calculate potential score

cus_score = []

for word_order in list_of_list_word:
    pos_score = 0
    neg_score = 0
    for word in word_order:
        if word in pos:
            pos_score += 1
        elif word in neg:
            neg_score +=1
    if pos_score > 0 and neg_score == 0:
        status = "Potential"
    elif neg_score < 0 and pos_score == 0:
        status = "Unpotential"
    elif pos_score - neg_score > 3:
        status = "Positive"
    elif neg_score - pos_score >3:
        status = "Negative"
    elif pos_score - neg_score <=3:
        status = "Neutral_Positive"
    elif neg_score - pos_score <=3:
        status = "Neutral_Negative"
    else:
        status = "Neutral"
    cus_score.append(status)

num = 0
while num < len(cus_score):
    word_file_3.update({word_file_2[num][0]:[cus_score[num],word_file_2[num][1]]})
    num += 1

In [60]:
with open("result.csv", "w") as f:
    writer = csv.DictWriter(f, fieldnames= ["Name","Result","Contain"])
    writer.writeheader()
    for key in word_file_3.keys():
        f.writelines("%s,%s,%s\n"%(key, word_file_3[key][0],word_file_3[key][1]))